In [1]:
import ihop.utils
spark = ihop.utils.get_spark_session("baseline lda")

input_data = spark.read.load("../data/bagOfWords/2021-05_to_2021-06_joined_submissions_comments_2percentTopUsersExcludedFromComments_02102022.parquet").limit(100)


22/02/19 14:57:13 WARN Utils: Your hostname, Kurt resolves to a loopback address: 127.0.1.1; using 192.168.0.11 instead (on interface wlp4s0)
22/02/19 14:57:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/19 14:57:19 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/02/19 14:57:19 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


Spark configuration:
[('spark.app.startTime', '1645300635734'), ('spark.app.id', 'local-1645300640094'), ('spark.executor.id', 'driver'), ('spark.app.name', 'baseline lda'), ('spark.driver.memory', '8G'), ('spark.driver.host', '192.168.0.11'), ('spark.sql.warehouse.dir', 'file:/home/virginia/Documents/CenterForDataScience/ZuckermanProj/IHOP/notebooks/spark-warehouse'), ('spark.rdd.compress', 'True'), ('spark.driver.port', '36873'), ('spark.serializer.objectStreamReset', '100'), ('spark.master', 'local[*]'), ('spark.submit.pyFiles', ''), ('spark.submit.deployMode', 'client'), ('spark.executor.extraLibraryPath', '/home/virginia/hadoop-3.3.1/lib/native'), ('spark.ui.showConsoleProgress', 'true'), ('spark.driver.extraLibraryPath', '/home/virginia/hadoop-3.3.1/lib/native')]


In [2]:
input_data.show(10)

+---------+------+-----------+-------------+------+------+------+---+-------+-------+-----+--------+-----+---+-----------+------------------+-----------+---------+--------------+-------+---------------+----+--------------------+--------------------------+
|subreddit|author|created_utc|distinguished|domain|edited|gilded| id|is_self|over_18|score|selftext|title|url|fullname_id|comments_subreddit|comments_id|parent_id|comments_score|link_id|comments_author|body|comments_created_utc|time_to_comment_in_seconds|
+---------+------+-----------+-------------+------+------+------+---+-------+-------+-----+--------+-----+---+-----------+------------------+-----------+---------+--------------+-------+---------------+----+--------------------+--------------------------+
+---------+------+-----------+-------------+------+------+------+---+-------+-------+-----+--------+-----+---+-----------+------------------+-----------+---------+--------------+-------+---------------+----+--------------------+----

In [4]:
input_data.count()

0

# Train a simple LDA model using Gensim


In [8]:
from datetime import datetime
import ihop.clustering as ic
import ihop.text_processing as itp

pipeline = itp.SparkTextPreprocessingPipeline('document_text')
corpus = itp.SparkCorpus.init_from_joined_dataframe(input_data)
transformed = pipeline.fit_transform(corpus.document_dataframe)
vectorized_corpus = itp.SparkCorpus(transformed)


22/02/19 15:16:03 ERROR Executor: Exception in task 0.0 in stage 5.0 (TID 4)
java.io.FileNotFoundException: 
File file:/home/virginia/Documents/CenterForDataScience/ZuckermanProj/IHOP/data/bagOfWords/2021-05_to_2021-06_joined_submissions_comments_2percentTopUsersExcludedFromComments_02102022.parquet/part-00000-tid-6790358003502038060-13b7671a-58f8-4459-b1c9-affd410a092d-1268-1-c000.snappy.parquet does not exist

It is possible the underlying files have been updated. You can explicitly invalidate
the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by
recreating the Dataset/DataFrame involved.
       
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:506)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:119)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScan

Py4JJavaError: An error occurred while calling o93.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 5.0 failed 1 times, most recent failure: Lost task 0.0 in stage 5.0 (TID 4) (192.168.0.11 executor driver): java.io.FileNotFoundException: 
File file:/home/virginia/Documents/CenterForDataScience/ZuckermanProj/IHOP/data/bagOfWords/2021-05_to_2021-06_joined_submissions_comments_2percentTopUsersExcludedFromComments_02102022.parquet/part-00000-tid-6790358003502038060-13b7671a-58f8-4459-b1c9-affd410a092d-1268-1-c000.snappy.parquet does not exist

It is possible the underlying files have been updated. You can explicitly invalidate
the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by
recreating the Dataset/DataFrame involved.
       
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:506)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$

In [ ]:
index = pipeline.get_id_to_word()
print("INDEX DETAILS:")
print("Vocab size:", len(index))
for k in range(10):
    print(k, index[k])

In [ ]:
vectorized_corpus.documents_dataframe.show()

In [ ]:
corpus_iterator = vectorized_corpus.get_vectorized_column_iterator()

In [ ]:
lda_model = ic.GensimLDAModel(corpus_iterator, "sample_lda", index, num_topics=10)

print("Starting training at", datetime.now())
lda.train()